extract hands data

In [1]:
# import lib
import mediapipe as mp
import cv2 as cv
import json
import os
import time
import gc
import numpy as np

In [2]:
# construct mp solutions

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
mp_hand = mp.solutions.hands

In [3]:
# setup data collect

sign_data = dict()

# structure **deprecated
# sign: [frame:
#           pose:
#               landmark_idx:
#                   x: float
#                   y: float
#                   z: float
#           ,right_hand:
#               landmark_idx:
#                   x: float
#                   y: float
#                   z: float
#           ,left_hand:
#               landmark_idx:
#                   x: float
#                   y: float,
#                   z: float]

In [9]:
# setup loop

# Specify the directory
directory = '../sibi/'

# Get a list of all files in the directory
signs = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# signs.sort(reverse=True)

# print(len(signs))

# signs = ['Ipar [Ipar].webm', 'A [A].webm', 'B [B].webm']

hand_landmarker = mp_hand.Hands(static_image_mode=True, min_detection_confidence=0.7, min_tracking_confidence=0.7,
                                model_complexity=0, max_num_hands=2)

start_time = time.time()
sign_meta = {}

In [10]:
directory = '../data/raw/hands'
extracted = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
print(len(signs), len(extracted))
signs = signs[2253:]

3394 3394


In [6]:
# loop through every file

print("- Extract Raw Data -")
for i in range(len(signs)):
    start_per_sign_time = time.time()
    sign = signs[i] # .split('[')[1][0:-6]
    sign_name = sign.split('[')[1][0:-6]

    print(f"File: {sign}")

    vid = '../sibi/' + sign
    cap = cv.VideoCapture(vid)

    current_frame = 0
    frames = []
    while cap.isOpened():
        ret, image = cap.read()
        if not ret:
            break

        image = cv.flip(image, 1)

        upscaled_img = cv.resize(image, None, fx=2, fy=2, interpolation=cv.INTER_CUBIC)

        frame = cv.cvtColor(upscaled_img, cv.COLOR_BGR2RGB)
        frame.flags.writeable = False
        hands_results = hand_landmarker.process(frame)
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        # hands
        hands_world = hands_results.multi_hand_world_landmarks
        temp_right_hand = {}
        temp_left_hand = {}
        temp_score = 0
        temp_handedness = ""
        if hands_world:
            for index in range(len(hands_world)):
                handedness = hands_results.multi_handedness[index].classification[0]
                handedness_score = handedness.score
                handedness_label = handedness.label

                if handedness.label == "Right":
                    if (handedness_score < temp_score) and (handedness_label == temp_handedness):
                        continue

                    temp_score = handedness_score
                    temp_handedness = handedness_label

                    for idx, landmark in enumerate(hands_world[index].landmark):
                        xyz = {"x": landmark.x,
                               "y": -landmark.y,
                               "z": -landmark.z,
                               "score": handedness_score}
                        temp_right_hand[idx] = xyz

                else:
                    if (handedness_score < temp_score) and (handedness_label == temp_handedness):
                        continue

                    temp_score = handedness_score
                    temp_handedness = handedness_label

                    for idx, landmark in enumerate(hands_world[index].landmark):
                        xyz = {"x": landmark.x,
                               "y": -landmark.y,
                               "z": -landmark.z,
                               "score": handedness_score}
                        temp_left_hand[idx] = xyz
        
        # into one
        frames.append({
            "frame_idx": current_frame,
            "right_hand": temp_right_hand,
            "left_hand": temp_left_hand})

        current_frame += 1

        del temp_right_hand, temp_left_hand

    end_per_sign_time = time.time()
    delta_per_sign_time = end_per_sign_time - start_per_sign_time
    sign_meta[sign] = {"time": delta_per_sign_time,
                       "frame_count": current_frame}

    print(f"Time: {delta_per_sign_time} seconds\n")

    sign_frame = {
        'metadata': {
            'sign': sign_name,
            'time': delta_per_sign_time,
            'frame_count': current_frame
        },
        'frames': frames
    }

    with open(f"../data/raw/hands/{sign_name}.json", "w") as json_file:
        json_file.write(json.dumps(sign_frame, indent=4))
    json_file.close()

    del frames

    if i % 500 == 0:
        gc.collect()

end_time = time.time()
delta_time = end_time - start_time
print(f"Finished in {delta_time} seconds.")

- Extract Raw Data -
File: Pertama [Pertama].webm
Time: 4.9532246589660645 seconds

File: Perunggu [Perunggu].webm
Time: 6.048591375350952 seconds

File: Perut [Perut].webm
Time: 5.761139631271362 seconds

File: Pesan [Pesan].webm
Time: 6.4128577709198 seconds

File: Pesat [Pesat].webm
Time: 6.2546820640563965 seconds

File: Pesawat [Pesawat].webm
Time: 4.9479100704193115 seconds

File: Pesek [Pesek].webm
Time: 7.358093976974487 seconds

File: Pesiar [Pesiar].webm
Time: 7.26454496383667 seconds

File: Pesimis [Pesimis].webm
Time: 6.20183253288269 seconds

File: Pesing [Pesing].webm
Time: 5.556010484695435 seconds

File: Pesona [Pesona].webm
Time: 6.710348844528198 seconds

File: Pesta [Pesta].webm
Time: 6.721632480621338 seconds

File: Peta [Peta].webm
Time: 7.535621166229248 seconds

File: Petaka [Petaka].webm
Time: 8.146151304244995 seconds

File: Petang [Petang].webm
Time: 7.085777997970581 seconds

File: Petasan [Petasan].webm
Time: 7.085189580917358 seconds

File: Peti [Peti].webm

In [7]:
# print(sign_meta)
# print(delta_time)
meta_data = {
    "total_time": delta_time,
    "meta": sign_meta,
}

with open(f"../data/meta/meta_data_raw_hands.json", "w") as meta_data_file:
    meta_data_file.write(json.dumps(meta_data, indent=4))
meta_data_file.close()